In [17]:
import pandas as pd
df = pd.read_csv('C:/ML_project_predict_heart_disease/data/heart_2020_final.csv')
X = df.drop(columns = 'HeartDisease').values
y = df['HeartDisease']

In [22]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE

def fold_K(X, y, model):    
    kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    acc_test_score = []
    acc_train_score = []
    rec_test_score = []
    rec_train_score = []
    
    for train_index, test_index in kf.split(X): # 5번
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_train = model.predict(X_train)

        acc_train_score.append(accuracy_score(y_train,y_pred_train))
        acc_test_score.append(accuracy_score(y_test,y_pred))
        
        rec_train_score.append(recall_score(y_train , y_pred_train))
        rec_test_score.append(recall_score(y_test , y_pred))

        
    print('정확도 : train score : {}'.format(np.array(acc_train_score).mean()))
    print('정확도 : test score : {}'.format(np.array(acc_test_score).mean()))
    # print('재현율 : train score : {}'.format(np.array(rec_train_score).mean()))
    # print('재현율 : test score : {}'.format(np.array(rec_test_score).mean()))
    


In [23]:
lgbm = LGBMClassifier(random_state = 42)
fold_K(X, y, lgbm)

[[218352   1210]
 [ 19718   2093]]
[[54537   357]
 [ 5022   428]]
[[218278   1140]
 [ 19992   1963]]
[[54694   344]
 [ 4841   465]]
[[218493   1073]
 [ 19913   1895]]
[[54578   312]
 [ 5029   424]]
[[218519   1136]
 [ 19751   1968]]
[[54477   324]
 [ 5104   438]]
[[218563   1060]
 [ 19838   1913]]
[[54533   300]
 [ 5076   434]]
정확도 : train score : 0.9131379732256774
정확도 : test score : 0.9114766420727254
재현율 : train score : 0.09016550287187867
재현율 : test score : 0.08034456644556309


In [16]:
xgb = XGBClassifier(random_state = 42)
fold_K(X, y, xgb)

정확도 : train score : 0.9107640526878067
정확도 : test score : 0.9169329203898815
재현율 : train score : 0.13563735524083484
재현율 : test score : 0.09750750334473024


In [ ]:
cat = CatBoostClassifier(randome_state = 42)
fold_k(X, y, cat)